In [ ]:
# chapter 2-4 webページを取得する

In [ ]:
from urllib.request import urlopen

In [ ]:
f = urlopen('http://sample.scraping-book.com/dp')

In [ ]:
type(f)

In [ ]:
f.read()

In [ ]:
f.status

In [ ]:
f.getheader('Content-Type')

In [ ]:
%%writefile urlopen_encoding.py

import sys
from urllib.request import urlopen

f = urlopen('http://sample.scraping-book.com/dp')
encoding = f.info().get_content_charset(failobj="utf-8")
print('encoding:', encoding, file=sys.stderr)

text = f.read().decode(encoding)
print(text)

In [ ]:
!python urlopen_encoding.py

In [ ]:
!python urlopen_encoding.py > dp.html

In [ ]:
%%writefile urlopen_meta.py

import re
import sys
from urllib.request import urlopen

f = urlopen('http://sample.scraping-book.com/dp')
bytes_content = f.read()

scanned_text = bytes_content[:1024].decode('ascii', errors='replace')

match = re.search(r'charset=["\']?([\w-]+)', scanned_text)
if match:
    encoding = match.group(1)
else:
    encoding = 'utf-8'

print('encoding:', encoding, file=sys.stderr)

text = bytes_content.decode(encoding)
print(text)

In [ ]:
!python urlopen_meta.py

In [ ]:
# chapter 2-5 webページからデータを抜き出す

In [ ]:
# 正規表現によるスクレイピング

In [ ]:
import re

In [ ]:
re.search(r'a.*c', 'abc123DEF')

In [ ]:
re.search(r'a.*d', 'abc123DEF')

In [ ]:
re.search(r'a.*d', 'abc123DEF', re.IGNORECASE)

In [ ]:
m = re.search(r'a(.*)c', 'abc123DEF')

In [ ]:
m.group(0)

In [ ]:
m.group(1)

In [ ]:
re.findall(r'\w{2,}', 'This is a pen')

In [ ]:
re.sub(r'\w{2,}', 'That', 'This is a pen')

In [ ]:
%%writefile scrape_re.py

import re
from html import unescape

with open('dp.html') as f:
    html = f.read()
    
for partial_html in re.findall(r'<a itemprop="url".*?</ul>\s*</a></li>', html, re.DOTALL):
    url = re.search(r'<a itemprop="url" href="(.*?)">', partial_html).group(1)
    url = 'http://sample.scraping-book.com' + url
    
    title = re.search(r'<p itemprop="name".*?</p>', partial_html).group(0)
    title = title.replace('<br/>', ' ')
    title = re.sub(r'<.*?>', '', title)
    title = unescape(title)
    
    print(url, title)

In [ ]:
!python scrape_re.py

In [ ]:
# XML(RSS)のスクレイピング

In [1]:
!wget http://gihyo.jp/feed/rss2 -O rss2.xml

--2017-08-23 23:08:37--  http://gihyo.jp/feed/rss2
Resolving gihyo.jp... 104.20.34.31, 104.20.33.31, 2400:cb00:2048:1::6814:211f, ...
Connecting to gihyo.jp|104.20.34.31|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/xml]
Saving to: ‘rss2.xml’

rss2.xml                [ <=>                ]  13.33K  --.-KB/s    in 0.08s   

2017-08-23 23:08:37 (168 KB/s) - ‘rss2.xml’ saved [13646]



In [4]:
%%writefile scrape_rss.py

from xml.etree import ElementTree

tree = ElementTree.parse('rss2.xml')
root = tree.getroot()

for item in root.findall('channel/item'):
    title = item.find('title').text
    url = item.find('link').text
    print(url, title)

Overwriting scrape_rss.py


In [5]:
!python scrape_rss.py

http://gihyo.jp/lifestyle/clip/01/everyday-cat/201708/23 2017年8月23日　座るしろくろこ ── 技評ねこ部通信
http://gihyo.jp/admin/serial/01/ubuntu-recipe/0485 第485回　aptlyで本格的なパッケージリポジトリを作る ── Ubuntu Weekly Recipe
http://gihyo.jp/dev/serial/01/perl-hackers-hub/004403 第44回　LINE Messaging APIで作るchatbot―LINE::Bot::APIとngrokでお手軽に！（3） ── Perl Hackers Hub
http://gihyo.jp/lifestyle/clip/01/everyday-cat/201708/22 2017年8月22日　宮古島のしろくろこ ── 技評ねこ部通信
http://gihyo.jp/design/serial/01/ui-design-unsung/0002 第2回　エラーと確認―スムーズな手続きを実現するには ── 縁の下のUIデザイン―少しの工夫で大きな改善！
http://gihyo.jp/dev/serial/01/perl-hackers-hub/004402 第44回　LINE Messaging APIで作るchatbot―LINE::Bot::APIとngrokでお手軽に！（2） ── Perl Hackers Hub
http://gihyo.jp/dev/serial/01/mysql-road-construction-news/0052 第52回　MySQLのパーティショニング機能 ── MySQL道普請便り
http://gihyo.jp/lifestyle/clip/01/everyday-cat/201708/21 2017年8月21日　見上げてごらん。しろくろこを！ ── 技評ねこ部通信
http://gihyo.jp/lifestyle/serial/01/engineer-x-manage/0003 第3回　チームづくりにおけるエンジニア出身社長の強みとは ── エンジニアと経営のクロスオーバー
http://gihyo.jp/dev/serial/01/

In [6]:
# chapter 2-6 データを保存する

In [7]:
# CSV形式での保存

In [8]:
%%writefile save_csv_join.py

print('rank,city,population')

print(','.join(['1', '上海', '24150000']))
print(','.join(['2', 'カラチ', '23500000']))
print(','.join(['3', '北京', '21516000']))
print(','.join(['4', '天津', '14722100']))
print(','.join(['5', 'イスタンブル', '14160467']))

Writing save_csv_join.py


In [9]:
!python save_csv_join.py

rank,city,population
1,上海,24150000
2,カラチ,23500000
3,北京,21516000
4,天津,14722100
5,イスタンブル,14160467


In [10]:
!python save_csv_join.py > top_cities.csv

In [13]:
%%writefile save_csv.py

import csv

with open('top_cities_2.csv', 'w', newline='')as f:
    writer = csv.writer(f)
    writer.writerow(['rank', 'city', 'population'])
    writer.writerows([
        [1, '上海', 24150000],
        [2, 'カラチ', 23500000],
        [3, '北京', 21516000],
        [4, '天津', 14722100],
        [5, 'イスタンブル', 14160467],
    ])

Overwriting save_csv.py


In [14]:
!python save_csv.py

In [23]:
%%writefile save_csv_dict.py

import csv

with open('top_cities_3.csv', 'w', newline='') as f:
    writer = csv.DictWriter(f, ['rank', 'city', 'population'])
    writer.writeheader()
    writer.writerows([
        {'rank': 1, 'city': '上海', 'population': 24150000},
        {'rank': 2, 'city': 'カラチ', 'population': 23500000},
        {'rank': 3, 'city': '北京', 'population': 21516000},
        {'rank': 4, 'city': '天津', 'population': 14722100},
        {'rank': 5, 'city': 'イスタンブル', 'population': 14160467},
    ])

Overwriting save_csv_dict.py


In [16]:
!python save_csv_dict.py

In [18]:
# デフォルト(環境依存)のエンコーディングの値を取得する
import locale

locale.getpreferredencoding()

'UTF-8'

In [19]:
# JSON形式での保存

In [20]:
%%writefile save_json.py

import json

cities = [
    {'rank': 1, 'city': '上海', 'population': 24150000},
    {'rank': 2, 'city': 'カラチ', 'population': 23500000},
    {'rank': 3, 'city': '北京', 'population': 21516000},
    {'rank': 4, 'city': '天津', 'population': 14722100},
    {'rank': 5, 'city': 'イスタンブル', 'population': 14160467},
]

print(json.dumps(cities, ensure_ascii=False, indent=2))

Writing save_json.py


In [21]:
!python save_json.py

[
  {
    "rank": 1,
    "city": "上海",
    "population": 24150000
  },
  {
    "rank": 2,
    "city": "カラチ",
    "population": 23500000
  },
  {
    "rank": 3,
    "city": "北京",
    "population": 21516000
  },
  {
    "rank": 4,
    "city": "天津",
    "population": 14722100
  },
  {
    "rank": 5,
    "city": "イスタンブル",
    "population": 14160467
  }
]


In [24]:
%%writefile save_json_file.py

import json

cities = [
    {'rank': 1, 'city': '上海', 'population': 24150000},
    {'rank': 2, 'city': 'カラチ', 'population': 23500000},
    {'rank': 3, 'city': '北京', 'population': 21516000},
    {'rank': 4, 'city': '天津', 'population': 14722100},
    {'rank': 5, 'city': 'イスタンブル', 'population': 14160467},
]

with open('top_cities.json', 'w') as f:
    json.dump(cities, f)

Writing save_json_file.py


In [25]:
!python save_json_file.py

In [26]:
# データベース(SQLite3)への保存

In [28]:
%%writefile save_sqlite3.py

import sqlite3

conn = sqlite3.connect('top_cities.db')

c = conn.cursor()
c.execute('DROP TABLE IF EXISTS cities')
c.execute('''
    CREATE TABLE cities (
        rank integer,
        city text,
        population integer
    )
''')

# execute()メソッドの第2引数にはSQL文のパラメーターのリストを指定できる
# パラメーターで置き換える場所(プレースホルダー)は ? で指定する
c.execute('INSERT INTO cities VALUES (?, ?, ?)', (1, '上海', 24150000))

# パラメーターが辞書の場合、プレースホルダーは :キー名 で指定する
c.execute('INSERT INTO cities VALUES (:rank, :city, :population)',
         {'rank': 2, 'city': 'カラチ', 'population': 23500000})

# executemany()メソッドでは、複数のパラメーターをリストで指定できる
# パラメーターの数(ここでは3つ)のSQLを順に実行できる
c.executemany('INSERT INTO cities VALUES (:rank, :city, :population)', [
    {'rank': 3, 'city': '北京', 'population': 21516000},
    {'rank': 4, 'city': '天津', 'population': 14722100},
    {'rank': 5, 'city': 'イスタンブル', 'population': 14160467},
])

conn.commit()

c.execute('SELECT * FROM cities')
for row in c.fetchall():
    print(row)
    
conn.close()

Writing save_sqlite3.py


In [29]:
!python save_sqlite3.py

(1, '上海', 24150000)
(2, 'カラチ', 23500000)
(3, '北京', 21516000)
(4, '天津', 14722100)
(5, 'イスタンブル', 14160467)


In [31]:
!sqlite3 top_cities.db 'SELECT * FROM cities'

1|上海|24150000
2|カラチ|23500000
3|北京|21516000
4|天津|14722100
5|イスタンブル|14160467


In [32]:
# chapter 2-7 Pythonによるスクレイピングの流れ

In [6]:
%%writefile python_scraper.py

import re
import sqlite3
from urllib.request import urlopen
from html import unescape


def main():
    """
    メインの処理。fetch(), scrape(), save()の3つの関数を呼び出す。
    """
    
    html = fetch('http://sample.scraping-book.com/dp')
    books = scrape(html)
    save('books.db', books)


def fetch(url):
    """
    引数urlで与えられたURLのwebページを取得する。
    webページのエンコーディングはContent-Typeヘッダーから取得する。
    戻り値: str型のHTML
    """
    
    f = urlopen(url)
    # HTTPヘッダーからエンコーディングを取得する(明示されていない場合はutf-8とする)。
    encoding = f.info().get_content_charset(failobj="utf-8")
    html = f.read().decode(encoding) # 得られたエンコーディングを指定して文字列にデコードする。
    
    return(html)


def scrape(html):
    """
    引数htmlで与えられたHTMLから正規表現で書籍の情報を抜き出す。
    戻り値: 書籍(dict)のリスト
    """
    
    books = []
    for partial_html in re.findall(r'<a itemprop="url".*?</ul>\s*</a></li>', html, re.DOTALL):
        # 書籍のURLは itemprop="url" という属性を持つa要素のhref属性から取得する。
        url = re.search(r'<a itemprop="url" href="(.*?)">', partial_html).group(1)
        url = 'http://sample.scraping-book.com/dp' + url # / で始まっているのでドメイン名などを追加する。
        
        # 書籍のタイトルは itemprop="name" という属性を持つp要素から取得する。
        title = re.search(r'<p itemprop="name".*?</p>', partial_html).group(0)
        title = re.sub(r'<.*?>', '', title) # タグを取り除く。
        title = unescape(title) # 文字参照を元に戻す。
        
        books.append({'url': url, 'title': title})
        
    return(books)


def save(db_path, books):
    """
    引数booksで与えられた書籍のリストをSQLiteデータベースに保存する。
    データベースのパスは引数db_pathで与えられる。
    戻り値: なし
    """
    
    conn = sqlite3.connect(db_path) # データベースを開き、コネクションを取得する。
    
    c = conn.cursor() # カーソルを取得する。
    # execute()メソッドでSQL文を実行する。
    # このスクリプトを何回実行しても同じ結果になるようにするため、booksテーブルが存在する場合は削除する。
    c.execute('DROP TABLE IF EXISTS books')
    # booksテーブルを作成する。
    c.execute('''
        CREATE TABLE books (
            title text,
            url text
        )
    ''')
    
    # executemany()メソッドでは、複数のパラメーターをリストで指定できる。
    c.executemany('INSERT INTO books VALUES (:title, :url)', books)
    
    conn.commit() # 変数をコミット(保存)する。
    conn.close() # コネクションを閉じる。
    
    
# pythonコマンドで実行された場合にmain()関数を呼び出す。これはモジュールとして他のファイルから
# インポートされた時に、main()関数が実行されないようにするための、Pythonにおける一般的なイディオム。
if __name__ == '__main__':
    main()

Overwriting python_scraper.py


In [7]:
# 実行
!python python_scraper.py

In [9]:
# sqlite3コマンドで実行結果を確認
!sqlite3 books.db 'SELECT * FROM books'

これからはじめるプログラミング 作って覚える基礎の基礎|http://sample.scraping-book.com/dp/dp/ebook/2016/978-4-7741-8336-7
情報処理技術者試験 平成28年度【秋期】情報セキュリティマネジメント パーフェクトラーニング過去問題集|http://sample.scraping-book.com/dp/dp/ebook/2016/978-4-7741-8337-4
大人の自由時間（大人の自由時間mini） 水泳のきれいなカラダをつくる～スリムな逆三角形になる！ドライランドトレーニング|http://sample.scraping-book.com/dp/dp/ebook/2016/978-4-7741-8338-1
30レッスン 30レッスンで絶対合格！ Microsoft Office Specialist PowerPoint 2013 テキスト＋問題集|http://sample.scraping-book.com/dp/dp/ebook/2016/978-4-7741-8346-6
ゼロからはじめる ゼロからはじめる海外旅行でスマホ活用 スマートガイド|http://sample.scraping-book.com/dp/dp/ebook/2016/978-4-7741-8334-3
今すぐ使えるかんたんmini 今すぐ使えるかんたんminiCD&DVD 作成超入門［Windows 10対応版］|http://sample.scraping-book.com/dp/dp/ebook/2016/978-4-7741-8335-0
大きな字でわかりやすい 大きな字でわかりやすいワード2016入門|http://sample.scraping-book.com/dp/dp/ebook/2016/978-4-7741-8325-1
大きな字でわかりやすい 大きな字でわかりやすいエクセル2016 入門|http://sample.scraping-book.com/dp/dp/ebook/2016/978-4-7741-8322-0
今すぐ使えるかんたん 今すぐ使えるかんたんぜったいデキます！ デジカメ写真活用術［Windows 10対応版］|http://sample.scraping-book.com/d